
# 🎯 Streaming Package Optimization Model

This notebook provides a detailed, step-by-step explanation of the core optimization model, which is implemented in `backend/streaming_optimizer.py`.

The rest of the backend primarily handles data preprocessing, postprocessing, and a few additional functionalities. For example, it includes logic for handling features like "live" and "highlights," which adjust the cost of certain packages. 




## 📋 Sets and Parameters

### Sets:
- **Packages**: Set of streaming packages, indexed by $p$.
- **Games**: Set of games, indexed by $g$.
- **StartDates**: Set of possible subscription start dates, indexed by $d$.

### Parameters:
- **AdjustedCost\_Month**: Adjusted monthly subscription cost for package $p$, calculated as $Cost\_Month[p] + 1$.
- **AdjustedCost\_Year**: Adjusted yearly subscription cost for package $p$, calculated as $Cost\_Year[p] + 12$.
- **CoverageMonth**: Games covered by a rolling monthly subscription starting on $d$.
- **CoverageYear**: Games covered by a rolling yearly subscription starting on $d$.
- **P\_g**: Packages that can stream game $g$.



## 📊 Decision Variables

- $z\_month_{p, d} \in \{0, 1\}$: Binary variable indicating if package $p$ is activated for a monthly subscription starting on date $d$.
- $z\_year_{p, d} \in \{0, 1\}$: Binary variable indicating if package $p$ is activated for a yearly subscription starting on date $d$.



## 🎯 Objective Function

Minimize the total adjusted cost of subscriptions:
$$
\min \sum_{p \in \text{Packages}} \sum_{d \in \text{StartDates}} \big( \text{AdjustedCost\_Month}[p] \cdot z\_month_{p, d} + \text{AdjustedCost\_Year}[p] \cdot z\_year_{p, d} \big)
$$


# 🛑 Constraints

## 1. **Game Coverage** 
Each game $g$ must be covered by at least one package:

$$
\sum_{p \in \text{P\_g}[g]} \  \sum_{d \in \text{StartDates} \mid g \in \text{CoverageMonth}[d]} z\_month_{p, d} + 
\sum_{d \in \text{StartDates} \mid g \in \text{CoverageYear}[d]} z\_year_{p, d} \geq 1, \quad \forall g \in \text{Games}
$$

---

### **Explanation**

#### **Left-Hand Side (LHS)**:
The LHS represents all possible ways that game $g$ can be streamed:
   - **First term**:
     $$
     \sum_{p \in \text{P\_g}[g]} \ \sum_{d \in \text{StartDates} \mid g \in \text{CoverageMonth}[d]} z\_month_{p, d}
     $$
     sums up all **monthly subscriptions** that:
     - Belong to packages that can stream $g$ ($p \in \text{P\_g}[g]$).
     - Have start dates $d$ such that the monthly subscription window covers $g$ ($g \in \text{CoverageMonth}[d]$).

   - **Second term**:
     $$
     \sum_{p \in \text{P\_g}[g]} \sum_{d \in \text{StartDates} \mid g \in \text{CoverageYear}[d]} z\_year_{p, d}
     $$
     sums up all **yearly subscriptions** that:
     - Belong to packages that can stream $g$.
     - Have start dates $d$ such that the yearly subscription window covers $g$.

#### **Right-Hand Side (RHS)**:
- The RHS ensures that the sum of these possibilities is **at least 1**, guaranteeing that **at least one subscription (monthly or yearly)** is active to cover the game.

#### **For Each Game**:
- This constraint is applied to **every game** $g$ in the set of games.
- Each game must have at least **one package activated** in the required time window.



## 2. **Binary Constraints**
Decision variables must be binary:
$$
z\_month_{p, d}, z\_year_{p, d} \in \{0, 1\}, \quad \forall p \in \text{Packages}, \forall d \in \text{StartDates}
$$



## 🖥️ Implementation in Python

Below is the Python implementation of the model:

In [2]:
import pulp
from datetime import datetime, timedelta

def optimize_streaming_packages(packages, games, game_dates, C_month, C_year, P_g):
    # Filter out unavailable subscriptions
    filtered_C_month = {p: C_month[p] for p in C_month if p in packages}
    filtered_C_year = {p: C_year[p] for p in C_year if p in packages}

    # Increase costs by 1 to avoid multiple inclusions of free packages
    adjusted_C_month = {p: cost + 1 for p, cost in filtered_C_month.items()}
    adjusted_C_year = {p: cost + 12 for p, cost in filtered_C_year.items()}

    # Generate possible start dates for rolling subscriptions
    start_dates = sorted(set(game_dates.values()))

    # Compute coverage windows for rolling monthly and yearly subscriptions
    coverage_month = {d: [g for g, gd in game_dates.items() if d <= gd <= d + timedelta(days=30)] for d in start_dates}
    coverage_year = {d: [g for g, gd in game_dates.items() if d <= gd <= d + timedelta(days=365)] for d in start_dates}

    # Model
    model = pulp.LpProblem("Streaming_Package_Optimization", pulp.LpMinimize)

    # Decision variables
    z_month = {(p, d): pulp.LpVariable(f"z_month_{p}_{d.strftime('%Y-%m-%d')}", cat='Binary')
               for p in adjusted_C_month for d in start_dates}
    z_year = {(p, d): pulp.LpVariable(f"z_year_{p}_{d.strftime('%Y-%m-%d')}", cat='Binary')
              for p in adjusted_C_year for d in start_dates}

    # Objective function: Minimize total cost (with adjusted costs)
    model += pulp.lpSum(adjusted_C_month[p] * z_month[p, d] for p in adjusted_C_month for d in start_dates) +              pulp.lpSum(adjusted_C_year[p] * z_year[p, d] for p in adjusted_C_year for d in start_dates)

    # Constraints
    # 1. Game coverage
    for g in games:
        model += pulp.lpSum(z_month[p, d] for p in P_g[g] if p in adjusted_C_month for d in start_dates if g in coverage_month[d]) +                  pulp.lpSum(z_year[p, d] for p in P_g[g] if p in adjusted_C_year for d in start_dates if g in coverage_year[d]) >= 1

    # Solve the model
    status = model.solve(pulp.PULP_CBC_CMD())

    # Process results
    actual_total_cost = pulp.value(model.objective) - sum(len(start_dates) for p in adjusted_C_month) - sum(len(start_dates) for p in adjusted_C_year)
    results = {
        "status": pulp.LpStatus[status],
        "total_cost": actual_total_cost,
        "active_monthly_subscriptions": [],
        "active_yearly_subscriptions": []
    }

    for p in adjusted_C_month:
        for d in start_dates:
            if z_month[p, d].varValue is not None and z_month[p, d].varValue > 0:
                results["active_monthly_subscriptions"].append({"package": p, "start_date": d})
    for p in adjusted_C_year:
        for d in start_dates:
            if z_year[p, d].varValue is not None and z_year[p, d].varValue > 0:
                results["active_yearly_subscriptions"].append({"package": p, "start_date": d})

    return results

### Mickey-Mouse Data 🐭

| **Game** | **Date**       | **Packages Available** |
|----------|----------------|-------------------------|
| G1       | January 15, 2023 | P1, P2                 |
| G2       | February 20, 2023 | P1                    |
| G3       | March 25, 2023  | P2                     |


In [6]:
# Mock data
packages = ["P1", "P2"]  # Package IDs
games = ["G1", "G2", "G3"]  # Game IDs

# Game dates (when the games are played)
game_dates = {
    "G1": datetime(2023, 1, 15),
    "G2": datetime(2023, 2, 20),
    "G3": datetime(2023, 3, 25),
}

# Monthly subscription costs for each package
C_month = {
    "P1": 30,
    "P2": 20,
}

# Yearly subscription costs for each package
C_year = {
    "P1": 300,
    "P2": 200,
}

# Packages that can stream each game
P_g = {
    "G1": ["P1", "P2"],
    "G2": ["P1"],
    "G3": ["P2"],
}


results = optimize_streaming_packages(packages, games, game_dates, C_month, C_year, P_g)

# Print results
print("Optimization Status:", results["status"])
print("Total Cost:", results["total_cost"])
print("\nActive Monthly Subscriptions:")
for sub in results["active_monthly_subscriptions"]:
    print(f"  Package: {sub['package']}, Start Date: {sub['start_date']}")

print("\nActive Yearly Subscriptions:")
for sub in results["active_yearly_subscriptions"]:
    print(f"  Package: {sub['package']}, Start Date: {sub['start_date']}")

Optimization Status: Optimal
Total Cost: 61.0

Active Monthly Subscriptions:
  Package: P1, Start Date: 2023-02-20 00:00:00
  Package: P2, Start Date: 2023-01-15 00:00:00
  Package: P2, Start Date: 2023-03-25 00:00:00

Active Yearly Subscriptions:
